In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
#import scipy
import sklearn
import itertools as it
from itertools import cycle 
import os.path as op
import timeit 
import json
from matplotlib import animation
import matplotlib.font_manager as font_manager
from collections import namedtuple
#from functools import partial
#from pathlib import Path

In [2]:
# Set plotting style
plt.style.use('seaborn-white')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

### Now use MD Analysis to calculate no. of frames a center PEG residues and terminal PEG residue is with 4 Angstroms of BSA (0.9 nm restrained system)

Import MDAnalysis

In [5]:
from prot_polymer_analysis import get_protresd_list, aa_frmcount, grptwocnt_aa, gtwo_trjcnt 
from prot_polymer_analysis import frac_cont, bavg_frac_cnt, prot_poly_cntmovie, AA_list_org

In [6]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

### First table will be total fractional contacts and oligomer occupancy values for each Rg value 

#### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near water molecules in BSA/water simulation

Calculate the number of surface bsa residues from a 1 ns BSA/water simulation

In [7]:
#Units of Angstroms 
dmax = 4.0 

In [8]:
def middle_of_band(band_start, band_stop, plot_min=0, plot_max=60):
    half_way = (band_stop - band_start) / 2
    mid_band = band_start + half_way
    plot_fraction = (mid_band - plot_min) / (plot_max - plot_min)

    return plot_fraction

# 0.9 nm PEG unrestrained Rg 100 ns trajectory C2

Load the rg = 1.2 nm (3 PEG N = 20 oligomer/BSA system) 

In [10]:
# Set up the MD Simulation, Make sure you do gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb 
# -dump 0 -n bsaplga_nk.ndx to generate 
# a new pdb file that contains unique chain identifiers 
uC2_n20PEG_NR = mda.Universe("bsapeg_n20waterT2/C2_pegNoRes/0.9nmpeg_prodC2/bpeg_0.9nmNR.pdb"
                             , "bsapeg_n20waterT2/C2_pegNoRes/0.9nmpeg_prodC2/C2NR_0.9nmbpeg.xtc")

In [11]:
uC2_n20PEG_NR

<Universe with 9665 atoms>

Check that we are on the first frame

In [12]:
uC2_n20PEG_NR.trajectory.frame

0

In [13]:
pn20C2_lenNR = len(uC2_n20PEG_NR.trajectory)
pn20C2_lenNR

10001

In [14]:
# Select one polymer chain, heavy atoms only 
#all_pn20 = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

#Select all the PLGA residues, heavy atoms only
allC2_n20PEG_NR = uC2_n20PEG_NR.select_atoms("resname sPEG PEG tPEG and not type H")
#list(all_n20PEG_NR)

In [15]:
# Select BSA residues, heavy atoms only 
protC2_09nmNR = uC2_n20PEG_NR.select_atoms("protein and not type H")
protC2_09nmNR

<AtomGroup with 4653 atoms>

Calculate AA frame counts for PLGA residues, 1.2 nm RG restraint, 100ns trajectory 

In [16]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20C2_lenNR - 1
s_time = timeit.default_timer()
h2diC2_09nmNR = aa_frmcount(protC2_09nmNR, allC2_n20PEG_NR, dmax, uC2_n20PEG_NR, start, end)
timeit.default_timer() - s_time

301.1240669442341

In [17]:
len(h2diC2_09nmNR.keys())

212

In [19]:
pr_res_PEG = list(protC2_09nmNR.residues)
ss_res_PEG = [str(row) for row in pr_res_PEG]
rkgC2_NR = {key:h2diC2_09nmNR[key][1] for key, value in h2diC2_09nmNR.items()}
plgC2_09nmNR_occ = pd.DataFrame(data=ss_res_PEG, columns=["BSA_des_res"])
plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'] = plgC2_09nmNR_occ['BSA_des_res'].map(rkgC2_NR)
plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'] = plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'].replace('nan', np.nan).fillna(0)
plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'] = plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'].round(2)
plgC2_09nmNR_occ

,BSA_des_res,mda_occ_0.9nmNRC2
0,"<Residue ASP, 1>",0.01
1,"<Residue THR, 2>",0.06
2,"<Residue HIS, 3>",0.04
3,"<Residue LYS, 4>",0.01
4,"<Residue SER, 5>",0.00
...,...,...
578,"<Residue GLN, 579>",0.03
579,"<Residue THR, 580>",0.30
580,"<Residue ALA, 581>",0.08
581,"<Residue LEU, 582>",0.00


In [20]:
plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'][plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'] != 0]

0      0.01
1      0.06
2      0.04
3      0.01
5      0.01
       ... 
575    0.77
576    0.64
578    0.03
579    0.30
580    0.08
Name: mda_occ_0.9nmNRC2, Length: 149, dtype: float64

In [22]:
bsaC2_NR = np.array(list(protC2_09nmNR.resids)) # shape is 4652
mC2_occ_09NR = np.array(list(plgC2_09nmNR_occ['mda_occ_0.9nmNRC2'])) # shape is 583
mC2_occ = np.zeros(shape=(4653))
at_ind = np.where(bsaC2_NR[:-1] != bsaC2_NR[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    mC2_occ[nw_v:b] = mC2_occ_09NR[i]
    nw_v = at_in_nw[i+1] + 1 

In [23]:
np.nonzero(mC2_occ)

(array([   0,    1,    2, ..., 4636, 4637, 4638]),)

### Visualize Occupanct on protein 

In [24]:
protC2_09nmNR.occupancies = mC2_occ
protC2_09nmNR.occupancies

array([0.01, 0.01, 0.01, ..., 0.  , 0.  , 0.  ])

In [25]:
with mda.Writer("prot09nm_NoResC2.pdb") as pdb:
    pdb.write(protC2_09nmNR)

In [26]:
# Frame count and occupancy for each residue
#h2di

### Residue Importance: 0.9 nm unrestrained C2

In [27]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmapC2_09nmPEGNR = prot_poly_cntmovie(protC2_09nmNR, allC2_n20PEG_NR, dmax, uC2_n20PEG_NR, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [28]:
np.save('0.9nmPEG_NoResC2.npy', trjmapC2_09nmPEGNR)    # .npy extension is added if not given

In [ ]:
trjmapC2_09nmPEGNR = np.load('0.9nmPEG_NoRes.npy', allow_pickle=True)

In [29]:
trjmapC2_09nmPEGNR[0].shape

(583, 60)

In [30]:
np.sum(trjmapC2_09nmPEGNR[1000][0])

0.0

In [31]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [32]:
ppC2_09nmNR_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_09nmNR_ct[i][j] = np.sum(trjmapC2_09nmPEGNR[i][j])

In [33]:
ppC2_09nmtotNR = np.zeros(shape=(583))
for i in range(583):
    ppC2_09nmtotNR[i] = np.sum(ppC2_09nmNR_ct[:,i])
#pp_12nmtot

In [34]:
np.sum(ppC2_09nmtotNR)

531641.0

In [39]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, ppC2_09nmtotNR/np.sum(ppC2_09nmtotNR), align='center', color='#1D77CF',width=wid, alpha=0.4, label='0.9 nm PEG')
plt.title("BSA in water with PEG Unrestrained, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PEG contacts C2', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block C2

In [36]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [37]:
# This code chunk gets the BSA residues and their corresponding number in a pandas dataframe 
red_bsa = []
bh = np.arange(0,584)
for i in range(583):
    b_str = str(list(protC2_09nmNR.residues[i:i+1]))
    if str(bh[i+1]) in b_str: 
        red_bsa.append(str(b_str[10:13])+" "+str(bh[i+1]))

In [40]:
prC2_res = list(protC2_09nmNR.residues)
ssC2_res = [str(row) for row in prC2_res]
rkg = {key:h2diC2_09nmNR[key][0] for key, value in h2diC2_09nmNR.items()}
plgC2_09nmaaNR = pd.DataFrame(data=ssC2_res, columns=["BSA_des_res"])
plgC2_09nmaaNR['mda_plga_frm_0.9nmNRC2'] = plgC2_09nmaaNR['BSA_des_res'].map(rkg)
plgC2_09nmaaNR['BSA_des_res'] = red_bsa
plgC2_09nmaaNR['mda_plga_frm_0.9nmNRC2'] = plgC2_09nmaaNR['mda_plga_frm_0.9nmNRC2'].replace('nan', np.nan).fillna(0)
plgC2_09nmaaNR.tail()

,BSA_des_res,mda_plga_frm_0.9nmNRC2
578,GLN 579,263.0
579,THR 580,2969.0
580,ALA 581,822.0
581,LEU 582,17.0
582,ALA 583,24.0


In [42]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames (VMD)"]
wat_data = wat_data.drop("BSA_res_no", axis=1)

#pr_res = list(prot_09nm.residues)
#ss_res = [str(row) for row in pr_res]

wat_data['BSA_des_res'] = ssC2_res
wat_data = wat_data[['BSA_des_res',"No. of frames (VMD)"]]
#wat_data.head()

# load MDAnalysis values from MDA_BSA1ns.txt file(129003 atoms SOL group was used to calc. frame counts for txt.
# file)
h2ob_dict = json.load(open("MDA_BSA1ns.txt"))
wat_data['Mda_frames'] = wat_data['BSA_des_res'].map(h2ob_dict)


# From MD Analysis
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row['Mda_frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['Mda_frames'] == 1000:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))
    
#c_list

# From VMD
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
vmd_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row["No. of frames (VMD)"] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row["No. of frames (VMD)"] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    vmd_list.append(str(str(a_a[i])+"  "+str(count)))

# Main difference is that Alanine 583 is counted for all 1001 frames. It seems VMD is unable to calc dist for that res
#vmd_list

#hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
#polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
#neg_res = ['ASP', 'GLU']
#pos_res = ['ARG', 'HIS', 'LYS']
# aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
#all_res = [pos_res, neg_res, polar_res, hydrophobic_res]

# Put the AA count in a pandas dataframe 
dg , ji = AA_list_org(c_list)
aa_count = pd.DataFrame(data=dg, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=ji, index=None)
vmg, vmdj = AA_list_org(vmd_list)
n2lf = pd.Series(data=vmdj, index=None)
aa_count['No_of_surf_res (MDAnalysis)'] = new_lf
aa_count['No_of_surf_res (VMD)'] = n2lf

In [43]:
aplC2_list = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_09nmaaNR.iterrows():
    if r_pl['mda_plga_frm_0.9nmNRC2'] != 0:
        aplC2_list.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_lC2 = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(aplC2_list)):
        if r_a['Amino_acids'] in aplC2_list[i]:
            count += 1
    cpl_lC2.append(count)      
        
aa_count['pegC2_0.9nmNR_100ns'] = cpl_lC2
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),pegC2_0.9nmNR_100ns
0,LYS,59,59,23
1,ARG,23,23,12
2,HIS,16,16,7
3,ASP,40,40,18
4,GLU,59,59,28
5,SER,23,23,10
6,THR,29,29,17
7,CYS,24,24,12
8,TYR,19,19,7
9,ASN,14,14,5


In [44]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['pegC2_0.9nmNR_100ns'].sum()

212

In [45]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [46]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['pegC2_0.9nmNR_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.4353182751540041

Calculate mean occupancy and the standard deviation for 1.2 nm trajectory

Numpy mean and std function was used to calculate mean occupancy and std dev using occ values from aa_frmcount output

In [47]:
# Mean occupancy and std deviation 
ll_moC2 = [value[1] for key, value in h2diC2_09nmNR.items()]
print("Mean Occpancy (0.9 nm Rg): "+str(np.mean(ll_moC2)), "Occ. std. dev.: "+str(np.std(ll_moC2)))

Mean Occpancy (0.9 nm Rg): 0.11570754716981133 Occ. std. dev.: 0.18748667736221486


### Calc. fractional contacts for each AA group type 

In [48]:
cd_12nm = frac_cont(h2diC2_09nmNR)
cd_12nm

{'Negative': [41591.0, 46, 191680.26086956522, 0.1554447596322544],
 'Positive': [53760.0, 42, 271360.0, 0.22006173083472721],
 'Polar': [81661.0, 58, 298485.0344827586, 0.24205901133748192],
 'Hydrophobic': [68288.0, 66, 219349.33333333334, 0.17788323242473783],
 'Aromatic': [27365.0, 23, 252233.91304347824, 0.20455126577079857]}

In [ ]:
cd = frac_cont(h2diC2_09nmNR)
kklh = []
for key, value in cd.items():
    kklh.append(value[1])
# Must substract aromatic residues, since they are already counted
sum(kklh) - cd['Aromatic'][1]

In [49]:
no_surf = aa_count['No_of_surf_res (MDAnalysis)'].sum()
no_surf

487

In [50]:
fcntC2_rg09nmNR, prgrpC2_09nmNR, aaC2_matx09nmNR = bavg_frac_cnt(5, protC2_09nmNR, allC2_n20PEG_NR, dmax, 
                                                                 uC2_n20PEG_NR, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [51]:
fcntC2_rg09nmNR

{'Negative': array([0.16165618, 0.14231441, 0.16875101, 0.17628799, 0.15260511]),
 'Positive': array([0.20412171, 0.20534435, 0.20630048, 0.2607429 , 0.26617429]),
 'Polar': array([0.24311267, 0.20303977, 0.20890858, 0.31197189, 0.28365405]),
 'Hydrophobic': array([0.17878195, 0.19211655, 0.16962098, 0.14703514, 0.16500385]),
 'Aromatic': array([0.21232749, 0.25718492, 0.24641894, 0.10396209, 0.1325627 ]),
 'total_frac': array([0.27310062, 0.23819302, 0.2238193 , 0.1724846 , 0.22587269])}

In [52]:
fc09nmC2_meanNR = np.array([np.mean(fcntC2_rg09nmNR['Negative']), np.mean(fcntC2_rg09nmNR['Positive'])
                        ,np.mean(fcntC2_rg09nmNR['Polar']),np.mean(fcntC2_rg09nmNR['Hydrophobic'])
                        , np.mean(fcntC2_rg09nmNR['Aromatic'])])
fc09nmC2_meanNR

array([0.16032294, 0.22853675, 0.25013739, 0.17051169, 0.19049123])

In [53]:
fc09nmC2_stdNR = np.array([np.std(fcntC2_rg09nmNR['Negative']), np.std(fcntC2_rg09nmNR['Positive'])
                       ,np.std(fcntC2_rg09nmNR['Polar']),np.std(fcntC2_rg09nmNR['Hydrophobic'])
                       , np.std(fcntC2_rg09nmNR['Aromatic'])])
fc09nmC2_stdNR

array([0.0119295 , 0.0285736 , 0.04222373, 0.01494955, 0.06147485])

In [54]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc09nmC2_meanNR, yerr=fc09nmC2_stdNR, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 0.9 nm Rg restrained Config 2', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [55]:
# Average of total fraction of contacts
np.mean(fcntC2_rg09nmNR['total_frac'])

0.22669404517453798

In [57]:
# Std Deviation of total fraction of contacts
np.std(fcntC2_rg09nmNR['total_frac'])

0.03235243103282321

### Avg no. PEG residues per BSA AA residue group C2

In [58]:
prgrpC2_09nmNR

{'Negative': array([array([7.77      , 3.88948583]), array([8.4375    , 3.38113794]),
        array([8.5275    , 3.32358899]), array([8.8205    , 2.90056542]),
        array([9.273     , 3.37897484])], dtype=object),
 'Positive': array([array([24.5945   ,  6.6585336]), array([24.8805    ,  5.33406222]),
        array([29.673     ,  7.27152467]),
        array([33.262     ,  8.07064781]),
        array([37.3675    ,  6.31454224])], dtype=object),
 'Polar': array([array([18.488     ,  5.69700412]),
        array([15.465     ,  5.84780087]), array([20.655    ,  5.8754553]),
        array([22.9825    ,  5.11656073]),
        array([21.9545    ,  4.82767333])], dtype=object),
 'Hydrophobic': array([array([15.1135    ,  4.36596126]),
        array([18.3725    ,  5.24297089]),
        array([17.479     ,  5.14427439]), array([9.0885    , 3.72742106]),
        array([9.389     , 3.32741326])], dtype=object),
 'Aromatic': array([array([7.4885    , 2.81529177]), array([8.4925    , 3.41173618]),


In [60]:
mean_09nmC2 = np.zeros(shape=5)
std_09nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrpC2_09nmNR.items():
    mpl_09nmC2 = []
    var_09nmC2 = []
    for i in prgrpC2_09nmNR[str(key)].flat:
        mpl_09nmC2.append(i[0])
        var_09nmC2.append((i[1])**2)
    
    # calc frac cont averages
    mean_09nmC2[count] = np.mean(mpl_09nmC2)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_09nmC2[count] = np.std(mpl_09nmC2)
    # std_12nm[count] = np.sqrt(np.sum(var_12nm)/5)
    
    count += 1


In [61]:
mean_09nmC2

array([ 8.5657, 29.9555, 19.909 , 13.8885,  5.3134])

In [62]:
std_09nmC2

array([0.49319961, 4.90808391, 2.68161763, 3.94421276, 2.7493735 ])

In [64]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, mean_09nmC2, yerr=std_09nmC2, ecolor='black',capsize=5)
plt.title(r'No. of PEG residues 0.9 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PEG residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PEG residues')

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PEG contact map 0.9 nm Unrestrained", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.93, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_12nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_res.mp4',writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.1 nm PEG unrestrained Rg 100 ns trajectory C2

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

In [65]:
# Set up the MD Simulation
u11nmNR_n20PEGC2 = mda.Universe("bsapeg_n20waterT2/C2_pegNoRes/1.1nmpeg_prodC2/bpeg_1.1nmNR.pdb"
                                , "bsapeg_n20waterT2/C2_pegNoRes/1.1nmpeg_prodC2/C2NR_1.1nmbpeg.xtc")

In [66]:
u11nmNR_n20PEGC2

<Universe with 9665 atoms>

In [67]:
pn20C2_11nmNR = len(u11nmNR_n20PEGC2.trajectory)
pn20C2_11nmNR

10001

In [68]:
#Select all the PLGA residues, heavy atoms only 
n20PEG_11nmNRC2 = u11nmNR_n20PEGC2.select_atoms("resname sPEG PEG tPEG and not type H")
n20PEG_11nmNRC2

<AtomGroup with 183 atoms>

In [69]:
# Select BSA residues, heavy atoms only 
protNR_11nmPEGC2 = u11nmNR_n20PEGC2.select_atoms("protein and not type H")
protNR_11nmPEGC2

<AtomGroup with 4653 atoms>

### Contact Analysis

In [70]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20C2_11nmNR - 1
s_time = timeit.default_timer()
h2diC2_11nmNR = aa_frmcount(protNR_11nmPEGC2, n20PEG_11nmNRC2, dmax, u11nmNR_n20PEGC2, start, end)
timeit.default_timer() - s_time
#h2di_11nm

313.68020696844906

In [71]:
len(h2diC2_11nmNR.keys())

144

In [72]:
pr_res_PEG = list(protNR_11nmPEGC2.residues)
ss_res_PEG = [str(row) for row in pr_res_PEG]
rkgC2_11NR = {key:h2diC2_11nmNR[key][1] for key, value in h2diC2_11nmNR.items()}
plgC2_09nmNR_occ['mda_occ_1.1nmNRC2'] = plgC2_09nmNR_occ['BSA_des_res'].map(rkgC2_11NR)
plgC2_09nmNR_occ['mda_occ_1.1nmNRC2'] = plgC2_09nmNR_occ['mda_occ_1.1nmNRC2'].replace('nan', np.nan).fillna(0)
plgC2_09nmNR_occ['mda_occ_1.1nmNRC2'] = plgC2_09nmNR_occ['mda_occ_1.1nmNRC2'].round(2)
plgC2_09nmNR_occ

,BSA_des_res,mda_occ_0.9nmNRC2,mda_occ_1.1nmNRC2
0,"<Residue ASP, 1>",0.01,0.0
1,"<Residue THR, 2>",0.06,0.0
2,"<Residue HIS, 3>",0.04,0.0
3,"<Residue LYS, 4>",0.01,0.0
4,"<Residue SER, 5>",0.00,0.0
...,...,...,...
578,"<Residue GLN, 579>",0.03,0.0
579,"<Residue THR, 580>",0.30,0.0
580,"<Residue ALA, 581>",0.08,0.0
581,"<Residue LEU, 582>",0.00,0.0


In [75]:
pr_res11nmC2 = list(protNR_11nmPEGC2.residues)
ss_res11nmC2 = [str(row) for row in pr_res11nmC2]
rkgC2_11nmNR = {key:h2diC2_11nmNR[key][0] for key, value in h2diC2_11nmNR.items()}
plgC2_1_1nmaaNR = pd.DataFrame(data=ss_res11nmC2, columns=["BSA_des_res"])
plgC2_1_1nmaaNR['mda_plga_frm_1.1nmNRC2'] = plgC2_1_1nmaaNR['BSA_des_res'].map(rkgC2_11nmNR)
plgC2_1_1nmaaNR['BSA_des_res'] = red_bsa
plgC2_1_1nmaaNR['mda_plga_frm_1.1nmNRC2'] = plgC2_1_1nmaaNR['mda_plga_frm_1.1nmNRC2'].replace('nan', np.nan).fillna(0)
plgC2_1_1nmaaNR.head()

,BSA_des_res,mda_plga_frm_1.1nmNRC2
0,ASP 1,0.0
1,THR 2,0.0
2,HIS 3,0.0
3,LYS 4,0.0
4,SER 5,0.0


In [78]:
bsa11C2_NR = np.array(list(protNR_11nmPEGC2.resids)) # shape is 4652
mC2_occ_11nmNR = np.array(list(plgC2_09nmNR_occ['mda_occ_1.1nmNRC2'])) # shape is 583
mC2_occ_11New = np.zeros(shape=(4653))
at_indC2 = np.where(bsa11C2_NR[:-1] != bsa11C2_NR[1:])[0]
atC2_in_nw = np.sort(np.append([0,4653],at_indC2))
nw_v = 0
for i in range(583):
    b = atC2_in_nw[i+1] +1
    mC2_occ_11New[nw_v:b] = mC2_occ_11nmNR[i]
    nw_v = atC2_in_nw[i+1] + 1 

In [79]:
mC2_occ_11New[0:33]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [80]:
protNR_11nmPEGC2.occupancies[3089:3099]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [81]:
np.nonzero(mC2_occ_11New)

(array([1679, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689,
        1703, 1704, 1705, 1706, 1707, 1708, 1709, 1710, 1711, 1810, 1811,
        1812, 1813, 1814, 1815, 1816, 1817, 1818, 1824, 1825, 1826, 1827,
        1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838,
        1839, 1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849,
        1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860,
        1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871,
        1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882,
        2377, 2378, 2379, 2380, 2381, 2382, 2383, 2384, 2385, 2386, 2387,
        2388, 2389, 2390, 2391, 2392, 2393, 2394, 2395, 2396, 2397, 2398,
        2399, 2400, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409,
        2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420,
        2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429, 2430, 2431,
        2432, 2433, 2434, 2435, 2436, 

### Visualize Occupanct on protein 

In [82]:
protNR_11nmPEGC2.occupancies = mC2_occ_11New
protNR_11nmPEGC2.occupancies[0:33]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [84]:
with mda.Writer("prot11nm_NoResC2.pdb") as pdb:
    pdb.write(protNR_11nmPEGC2)

### Residue Importance: 1.1 nm unrestrained 

In [85]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmapC2_11nmPEGNR = prot_poly_cntmovie(protNR_11nmPEGC2, n20PEG_11nmNRC2, dmax, u11nmNR_n20PEGC2, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
trjmapC2_11nmPEGNR = np.load('1.1nmNR_PEGres.npy', allow_pickle=True)

In [86]:
np.save('1.1nmNR_PEGresC2.npy', trjmapC2_11nmPEGNR)    # .npy extension is added if not given

In [87]:
trjmapC2_11nmPEGNR[0].shape

(583, 60)

In [88]:
np.sum(trjmapC2_11nmPEGNR[1000][0])

0.0

In [89]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [90]:
ppC2_11nmNR_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_11nmNR_ct[i][j] = np.sum(trjmapC2_11nmPEGNR[i][j])

In [91]:
ppC2_11nmtotNR = np.zeros(shape=(583))
for i in range(583):
    ppC2_11nmtotNR[i] = np.sum(ppC2_11nmNR_ct[:,i])
#pp_12nmtot

In [92]:
np.sum(ppC2_11nmtotNR)

573298.0

In [111]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, ppC2_11nmtotNR/np.sum(ppC2_11nmtotNR), align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
#plt.bar(y_pos, pp_09nmtotNR, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.title("BSA in water with PEG Unrestrained, 100 ns, Config 2", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block

In [94]:
aplC2_11nmNR = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_1_1nmaaNR.iterrows():
    if r_pl['mda_plga_frm_1.1nmNRC2'] != 0:
        aplC2_11nmNR.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cplC2_11nmNR = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(aplC2_11nmNR)):
        if r_a['Amino_acids'] in aplC2_11nmNR[i]:
            count += 1
    cplC2_11nmNR.append(count)      
        
aa_count['pegC2_1.1nmNR_100ns'] = cplC2_11nmNR
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),pegC2_0.9nmNR_100ns,pegC2_1.1nmNR_100ns
0,LYS,59,59,23,16
1,ARG,23,23,12,6
2,HIS,16,16,7,2
3,ASP,40,40,18,9
4,GLU,59,59,28,21
5,SER,23,23,10,5
6,THR,29,29,17,9
7,CYS,24,24,12,6
8,TYR,19,19,7,7
9,ASN,14,14,5,5


In [95]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['pegC2_1.1nmNR_100ns'].sum()

144

In [96]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [97]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['pegC2_1.1nmNR_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.29568788501026694

In [98]:
# Mean occupancy and std deviation 
ll_mo11 = [value[1] for key, value in h2diC2_11nmNR.items()]
print("Mean Occpancy (1.1 nm Rg): "+str(np.mean(ll_mo11)), "Occ. std. dev.: "+str(np.std(ll_mo11)))

Mean Occpancy (1.1 nm Rg): 0.17118472222222222 Occ. std. dev.: 0.2728237450767448


In [99]:
cd_11nmNRC2 = frac_cont(h2diC2_11nmNR)
cd_11nmNRC2

{'Negative': [36320.0, 30, 174336.0, 0.12575696993920146],
 'Positive': [60552.0, 24, 363312.0, 0.26207447837825326],
 'Polar': [62188.0, 38, 235659.7894736842, 0.16999277866143914],
 'Hydrophobic': [87446.0, 52, 242158.15384615384, 0.17468036248258131],
 'Aromatic': [41203.0, 16, 370827.0, 0.2674954105385248]}

### Calc. fractional contacts for each AA group type 

In [100]:
fcntC2_rg11nmNR, prgrpC2_11nmNR, aamatxC2_11nmNR = bavg_frac_cnt(5, protNR_11nmPEGC2, n20PEG_11nmNRC2,
                                                             dmax ,u11nmNR_n20PEGC2, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [101]:
fcntC2_rg11nmNR

{'Negative': array([0.07970126, 0.12365648, 0.11347921, 0.1291214 , 0.1627071 ]),
 'Positive': array([0.22594526, 0.25498242, 0.27193731, 0.24563133, 0.23838792]),
 'Polar': array([0.16933767, 0.21620801, 0.18876536, 0.18161038, 0.16079839]),
 'Hydrophobic': array([0.19660706, 0.15557993, 0.15928259, 0.15994885, 0.18997493]),
 'Aromatic': array([0.32840876, 0.24957316, 0.26653552, 0.28368804, 0.24813166]),
 'total_frac': array([0.13141684, 0.20944559, 0.20944559, 0.18685832, 0.16837782])}

In [102]:
fc11nmC2_meanNR = np.array([np.mean(fcntC2_rg11nmNR['Negative']), np.mean(fcntC2_rg11nmNR['Positive'])
                        ,np.mean(fcntC2_rg11nmNR['Polar']),np.mean(fcntC2_rg11nmNR['Hydrophobic'])
                        , np.mean(fcntC2_rg11nmNR['Aromatic'])])
fc11nmC2_meanNR

array([0.12173309, 0.24737685, 0.18334396, 0.17227867, 0.27526743])

In [103]:
fc11nmC2_stdNR = np.array([np.std(fcntC2_rg11nmNR['Negative']), np.std(fcntC2_rg11nmNR['Positive'])
                       ,np.std(fcntC2_rg11nmNR['Polar']),np.std(fcntC2_rg11nmNR['Hydrophobic'])
                       , np.std(fcntC2_rg11nmNR['Aromatic'])])
fc11nmC2_stdNR

array([0.0267281 , 0.01551916, 0.01906264, 0.01734819, 0.02955571])

In [110]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc09nmC2_meanNR, width, yerr=fc09nmC2_stdNR, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc11nmC2_meanNR, width, yerr=fc11nmC2_stdNR, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg restrained, Config 2', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [105]:
np.mean(fcntC2_rg11nmNR['total_frac'])

0.18110882956878846

In [106]:
np.std(fcntC2_rg11nmNR['total_frac'])

0.029224553387527222

### Avg no. PEG residues per BSA AA residue group C2

In [108]:
prgrpC2_11nmNR

{'Negative': array([array([3.5775    , 2.20045308]), array([7.332     , 2.87520017]),
        array([8.781     , 2.90981769]), array([7.597     , 2.81950191]),
        array([10.3195    ,  3.33847566])], dtype=object),
 'Positive': array([array([19.2025    ,  5.12449937]),
        array([33.637     ,  7.61066561]),
        array([35.751     ,  5.98949071]),
        array([31.297     ,  3.87385996]),
        array([39.616     ,  6.40215151])], dtype=object),
 'Polar': array([array([13.3115    ,  4.44617451]),
        array([26.3155    ,  5.16303784]),
        array([23.748     ,  5.37591815]),
        array([17.503     ,  4.46754866]),
        array([19.8775    ,  4.86995829])], dtype=object),
 'Hydrophobic': array([array([12.778     ,  4.89782768]),
        array([20.953     ,  5.73400305]),
        array([18.794     ,  4.81752675]),
        array([14.0655    ,  4.35180534]),
        array([21.99      ,  6.32707673])], dtype=object),
 'Aromatic': array([array([10.616     ,  3.28078405]

In [112]:
mean_11nmC2 = np.zeros(shape=5)
std_11nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrpC2_11nmNR.items():
    mpl_11nm = []
    var_11nm = []
    for i in prgrpC2_11nmNR[str(key)].flat:
        mpl_11nm.append(i[0])
        var_11nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_11nmC2[count] = np.mean(mpl_11nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_11nmC2[count] = np.std(mpl_11nm)
    #std_15nm[count] = np.sqrt(np.sum(var_15nm)/5)
    
    count += 1


In [113]:
mean_11nmC2

array([ 7.5214, 31.9007, 20.1511, 17.7161, 13.8189])

In [114]:
std_11nmC2

array([2.23659968, 6.91338034, 4.57831706, 3.67742636, 1.82262523])

In [115]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, mean_09nmC2, width, yerr=std_09nmC2, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_11nmC2, width, yerr=std_11nmC2, ecolor='black',capsize=5, color='c')
plt.title(r'No. of PEG residues Rg restrained, Config 2', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PLGA residues')

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm res.", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_15nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.4 nm unrestrained Rg PEG 100 ns trajectory C2

### Contact Analyis

In [116]:
# Set up the MD Simulation
u14nmNR_n20PEGC2 = mda.Universe("bsapeg_n20waterT2/C2_pegNoRes/1.4nmpeg_prodC2/bpeg_1.4nmNR.pdb"
                              , "bsapeg_n20waterT2/C2_pegNoRes/1.4nmpeg_prodC2/C2NR_1.4nmbpeg.xtc")

In [117]:
u14nmNR_n20PEGC2

<Universe with 9665 atoms>

In [118]:
pn20C2_len14nmNR = len(u14nmNR_n20PEGC2.trajectory)
pn20C2_len14nmNR

10001

In [119]:
#Select all the PLGA residues, heavy atoms only 
pn20C2_14nmNR = u14nmNR_n20PEGC2.select_atoms("resname sPEG PEG tPEG and not type H")
pn20C2_14nmNR

<AtomGroup with 183 atoms>

In [120]:
# Select BSA residues, heavy atoms only 
protC2_14nmNR = u14nmNR_n20PEGC2.select_atoms("protein and not type H")
protC2_14nmNR

<AtomGroup with 4653 atoms>

In [124]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20C2_len14nmNR - 1
s_time = timeit.default_timer()
h2diC2_14nmNR = aa_frmcount(protC2_14nmNR, pn20C2_14nmNR, dmax, u14nmNR_n20PEGC2, start, end)
timeit.default_timer() - s_time
#h2di_14nm

216.04537139181048

In [125]:
len(h2diC2_14nmNR.keys())

192

In [126]:
prC2_res14nmNR = list(protC2_14nmNR.residues)
ssC2_res14nmNR = [str(row) for row in prC2_res14nmNR]
rkgC2_14nmNR = {key:h2diC2_14nmNR[key][0] for key, value in h2diC2_14nmNR.items()}
plgC2_14nmaaNR = pd.DataFrame(data=ssC2_res14nmNR, columns=["BSA_des_res"])
plgC2_14nmaaNR['mda_plga_frm_1.4nmNRC2'] = plgC2_14nmaaNR['BSA_des_res'].map(rkgC2_14nmNR)
plgC2_14nmaaNR['BSA_des_res'] = red_bsa
plgC2_14nmaaNR['mda_plga_frm_1.4nmNRC2'] = plgC2_14nmaaNR['mda_plga_frm_1.4nmNRC2'].replace('nan', np.nan).fillna(0)
plgC2_14nmaaNR.head()

,BSA_des_res,mda_plga_frm_1.4nmNRC2
0,ASP 1,7001.0
1,THR 2,7038.0
2,HIS 3,5762.0
3,LYS 4,6819.0
4,SER 5,1024.0


In [127]:
# Extract mean occupancy values
pr_res_14ur = list(protC2_14nmNR.residues)
ss_res_14ur = [str(row) for row in pr_res_14ur]
rkgC2_14NR = {key:h2diC2_14nmNR[key][1] for key, value in h2diC2_14nmNR.items()}
plgC2_09nmNR_occ['mda_occ_1.4nmNRC2'] = plgC2_09nmNR_occ['BSA_des_res'].map(rkgC2_14NR)
plgC2_09nmNR_occ['mda_occ_1.4nmNRC2'] = plgC2_09nmNR_occ['mda_occ_1.4nmNRC2'].replace('nan', np.nan).fillna(0)
plgC2_09nmNR_occ['mda_occ_1.4nmNRC2'] = plgC2_09nmNR_occ['mda_occ_1.4nmNRC2'].round(2)
plgC2_09nmNR_occ

,BSA_des_res,mda_occ_0.9nmNRC2,mda_occ_1.1nmNRC2,mda_occ_1.4nmNRC2
0,"<Residue ASP, 1>",0.01,0.0,0.70
1,"<Residue THR, 2>",0.06,0.0,0.70
2,"<Residue HIS, 3>",0.04,0.0,0.58
3,"<Residue LYS, 4>",0.01,0.0,0.68
4,"<Residue SER, 5>",0.00,0.0,0.10
...,...,...,...,...
578,"<Residue GLN, 579>",0.03,0.0,0.05
579,"<Residue THR, 580>",0.30,0.0,0.47
580,"<Residue ALA, 581>",0.08,0.0,0.32
581,"<Residue LEU, 582>",0.00,0.0,0.09


In [129]:
bsa14C2_NR = np.array(list(protC2_14nmNR.resids)) # shape is 4652
mC2_occ_14nmNR = np.array(list(plgC2_09nmNR_occ['mda_occ_1.4nmNRC2'])) # shape is 583
mC2_occ_14New = np.zeros(shape=(4653))
at_ind = np.where(bsaC2_NR[:-1] != bsa14C2_NR[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    mC2_occ_14New[nw_v:b] = mC2_occ_14nmNR[i]
    nw_v = at_in_nw[i+1] + 1 

In [130]:
np.nonzero(mC2_occ_14New)

(array([   0,    1,    2, ..., 4650, 4651, 4652]),)

### Visualize Occupanct on protein 

In [131]:
protC2_14nmNR.occupancies = mC2_occ_14New
protC2_14nmNR.occupancies

array([0.7 , 0.7 , 0.7 , ..., 0.18, 0.18, 0.18])

In [132]:
with mda.Writer("prot14nm_NoResC2.pdb") as pdb:
    pdb.write(protC2_14nmNR)

### Residue Importance: 1.4 nm unrestrained 

In [133]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmapC2_14nmPEGNR = prot_poly_cntmovie(protC2_14nmNR, pn20C2_14nmNR, dmax, u14nmNR_n20PEGC2, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [134]:
np.save('1.4nmPEG_NoResC2.npy', trjmapC2_14nmPEGNR)    # .npy extension is added if not given

In [ ]:
trjmapC2_14nmPEGNR = np.load('1.4nm_NoRes.npy', allow_pickle=True)
trjmapC2_14nmPEGNR

In [135]:
trjmapC2_14nmPEGNR[0].shape

(583, 60)

In [136]:
np.sum(trjmapC2_14nmPEGNR[1000][0])

0.0

In [137]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [138]:
ppC2_14nmNR_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        ppC2_14nmNR_ct[i][j] = np.sum(trjmapC2_14nmPEGNR[i][j])

In [139]:
ppC2_14nmtotNR = np.zeros(shape=(583))
for i in range(583):
    ppC2_14nmtotNR[i] = np.sum(ppC2_14nmNR_ct[:,i])
#pp_12nmtot

In [140]:
np.nonzero(ppC2_14nmtotNR)

(array([  0,   1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  14,
         44,  47,  48,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
         61,  62,  63,  64,  66,  67,  93,  94,  95,  96,  98,  99, 102,
        103, 105, 110, 111, 112, 113, 114, 115, 146, 184, 201, 202, 203,
        204, 205, 206, 207, 210, 225, 226, 227, 228, 230, 231, 234, 235,
        238, 239, 241, 242, 244, 245, 247, 250, 253, 254, 257, 258, 260,
        261, 263, 264, 265, 268, 269, 284, 310, 312, 315, 316, 318, 319,
        320, 321, 322, 323, 324, 325, 327, 352, 396, 407, 408, 411, 414,
        415, 418, 419, 460, 463, 464, 465, 466, 470, 476, 477, 488, 490,
        491, 492, 493, 494, 495, 496, 497, 498, 499, 501, 502, 503, 504,
        505, 506, 507, 508, 509, 510, 511, 512, 514, 516, 518, 519, 520,
        522, 523, 529, 533, 534, 535, 536, 537, 538, 539, 540, 541, 543,
        544, 545, 547, 548, 550, 551, 552, 553, 554, 555, 556, 557, 558,
        559, 560, 561, 563, 564, 565, 566, 567, 568

In [141]:
a_peg = np.sum(ppC2_14nmtotNR)

In [142]:
b_peg = np.sum(ppC2_11nmtotNR)

In [143]:
c_peg = np.sum(ppC2_09nmtotNR)

In [ ]:
plt.close('all')

In [187]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
#plt.bar(y_pos, ppC2_09nmtotNR/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
#plt.bar(y_pos+0.25, ppC2_11nmtotNR/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
plt.bar(y_pos+0.3, ppC2_14nmtotNR/a_peg, align='center',width=wid, color='#4E4C4D', alpha=0.3, label='1.4 nm PEG')
#plt.bar(y_pos+0.25, pp_11nmtot/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
#plt.bar(y_pos, pp_09nmtot/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.title("BSA in water with PEG Unrestrained, 100 ns, Config. 2", fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=16)
plt.ylabel(r'Normalized Total No. of PEG contacts', fontsize=16)
plt.xlabel(r'BSA Residue ID', fontsize=16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block

In [145]:
apl_14nmNR = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plgC2_14nmaaNR.iterrows():
    if r_pl['mda_plga_frm_1.4nmNRC2'] != 0:
        apl_14nmNR.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_14nmNR = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_14nmNR)):
        if r_a['Amino_acids'] in apl_14nmNR[i]:
            count += 1
    cpl_14nmNR.append(count)      
        
aa_count['pegC2_1.4nmNR_100ns'] = cpl_14nmNR
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),pegC2_0.9nmNR_100ns,pegC2_1.1nmNR_100ns,pegC2_1.4nmNR_100ns
0,LYS,59,59,23,16,29
1,ARG,23,23,12,6,5
2,HIS,16,16,7,2,7
3,ASP,40,40,18,9,20
4,GLU,59,59,28,21,26
5,SER,23,23,10,5,6
6,THR,29,29,17,9,16
7,CYS,24,24,12,6,8
8,TYR,19,19,7,7,3
9,ASN,14,14,5,5,4


In [146]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['pegC2_1.4nmNR_100ns'].sum()

192

In [147]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [148]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['pegC2_1.4nmNR_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.3942505133470226

In [149]:
# Mean occupancy and std deviation 
ll_mo14nm = [value[1] for key, value in h2diC2_14nmNR.items()]
print("Mean Occpancy (1.4 nm Rg): "+str(np.mean(ll_mo14nm)), "Occ. std. dev.: "+str(np.std(ll_mo14nm)))

Mean Occpancy (1.4 nm Rg): 0.08028489583333333 Occ. std. dev.: 0.13628252605261182


In [150]:
cd_14NRnm = frac_cont(h2diC2_14nmNR)
cd_14NRnm

{'Negative': [36743.0, 46, 153362.08695652173, 0.19279509687403182],
 'Positive': [40621.0, 41, 190225.17073170733, 0.23913654897964448],
 'Polar': [42547.0, 44, 185659.63636363638, 0.23339710803800223],
 'Hydrophobic': [34236.0, 61, 107759.21311475411, 0.13546664852975243],
 'Aromatic': [15681.0, 19, 158460.6315789474, 0.19920459757856906]}

### Calc. fractional contacts for each AA group type 

In [151]:
fcntC2_rg14nmNR, prgrpC2_14nmNR, aa_matxC2_14nmNR = bavg_frac_cnt(5, protC2_14nmNR, pn20C2_14nmNR, dmax
                                                                  , u14nmNR_n20PEGC2, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [152]:
fcntC2_rg14nmNR

{'Negative': array([0.19133871, 0.13890481, 0.15207755, 0.17840408, 0.20347158]),
 'Positive': array([0.182228  , 0.20041814, 0.25378444, 0.22448271, 0.23714304]),
 'Polar': array([0.30865804, 0.26280296, 0.20509779, 0.28272453, 0.17685606]),
 'Hydrophobic': array([0.19519338, 0.16109075, 0.10962789, 0.107809  , 0.15987736]),
 'Aromatic': array([0.12258186, 0.23678335, 0.27941233, 0.20657968, 0.22265196]),
 'total_frac': array([0.20328542, 0.16632444, 0.11293634, 0.15811088, 0.21149897])}

In [153]:
fc14nmC2_meanNR = np.array([np.mean(fcntC2_rg14nmNR['Negative']), np.mean(fcntC2_rg14nmNR['Positive'])
                        ,np.mean(fcntC2_rg14nmNR['Polar']),np.mean(fcntC2_rg14nmNR['Hydrophobic'])
                        , np.mean(fcntC2_rg14nmNR['Aromatic'])])
fc14nmC2_meanNR

array([0.17283934, 0.21961127, 0.24722788, 0.14671968, 0.21360184])

In [154]:
fc14nmC2_stdNR = np.array([np.std(fcntC2_rg14nmNR['Negative']), np.std(fcntC2_rg14nmNR['Positive'])
                       ,np.std(fcntC2_rg14nmNR['Polar']),np.std(fcntC2_rg14nmNR['Hydrophobic'])
                       , np.std(fcntC2_rg14nmNR['Aromatic'])])
fc14nmC2_stdNR

array([0.02405881, 0.02555302, 0.04899673, 0.0335237 , 0.05154465])

In [176]:
plt.close("all")

In [177]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, fc09nmC2_meanNR, width, yerr=fc09nmC2_stdNR, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc11nmC2_meanNR, width, yerr=fc11nmC2_stdNR, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc14nmC2_meanNR, width, yerr=fc14nmC2_stdNR, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts Rg PEG/BSA Unrestrained, Config. 2', fontsize=16)
plt.xticks(x_pos+width, labels=aa_types, fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(0,0.4)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm', 'Rg = 1.4 nm'], frameon=False, fontsize=16)
plt.ylabel(r'Fractional Contacts', fontsize=16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [156]:
np.mean(fcntC2_rg14nmNR['total_frac'])

0.17043121149897328

In [157]:
np.std(fcntC2_rg14nmNR['total_frac'])

0.035327824505308525

In [158]:
prgrpC2_14nmNR

{'Negative': array([array([3.823     , 2.61470285]), array([8.297     , 5.30431815]),
        array([13.4635    ,  3.17988486]),
        array([12.6225    ,  2.82471127]),
        array([22.6535    ,  5.31765341])], dtype=object),
 'Positive': array([array([6.645     , 4.11132278]), array([11.212     ,  7.96530326]),
        array([18.9065    ,  4.50896415]),
        array([19.411     ,  5.55239399]),
        array([41.097     ,  9.15459398])], dtype=object),
 'Polar': array([array([8.2135    , 5.86156274]), array([16.5845    ,  4.98205377]),
        array([13.993    ,  4.7894625]), array([12.6065    ,  3.94368581]),
        array([19.041     ,  5.85126644])], dtype=object),
 'Hydrophobic': array([array([6.5215    , 3.67321899]), array([4.7525    , 2.49684676]),
        array([4.485     , 2.58104146]), array([3.5255    , 2.50166939]),
        array([12.7115    ,  6.76869764])], dtype=object),
 'Aromatic': array([array([0.755     , 1.14977172]), array([3.0495    , 2.62298489]),
        

### Avg no. PLGA residues per BSA AA residue group 

In [159]:
prgrpC2_14nmNR

{'Negative': array([array([3.823     , 2.61470285]), array([8.297     , 5.30431815]),
        array([13.4635    ,  3.17988486]),
        array([12.6225    ,  2.82471127]),
        array([22.6535    ,  5.31765341])], dtype=object),
 'Positive': array([array([6.645     , 4.11132278]), array([11.212     ,  7.96530326]),
        array([18.9065    ,  4.50896415]),
        array([19.411     ,  5.55239399]),
        array([41.097     ,  9.15459398])], dtype=object),
 'Polar': array([array([8.2135    , 5.86156274]), array([16.5845    ,  4.98205377]),
        array([13.993    ,  4.7894625]), array([12.6065    ,  3.94368581]),
        array([19.041     ,  5.85126644])], dtype=object),
 'Hydrophobic': array([array([6.5215    , 3.67321899]), array([4.7525    , 2.49684676]),
        array([4.485     , 2.58104146]), array([3.5255    , 2.50166939]),
        array([12.7115    ,  6.76869764])], dtype=object),
 'Aromatic': array([array([0.755     , 1.14977172]), array([3.0495    , 2.62298489]),
        

In [163]:
mean_14nmC2 = np.zeros(shape=5)
std_14nmC2 = np.zeros(shape=5)
count = 0
for key, value in prgrpC2_14nmNR.items():
    mpl_14nm = []
    var_14nm = []
    for i in prgrpC2_14nmNR[str(key)].flat:
        mpl_14nm.append(i[0])
        var_14nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_14nmC2[count] = np.mean(mpl_14nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_14nmC2[count] = np.std(mpl_14nm)
    #std_2nm[count] = np.sqrt(np.sum(var_2nm)/5)
    
    count += 1


In [164]:
mean_14nmC2

array([12.1719, 19.4543, 14.0877,  6.3992,  4.907 ])

In [165]:
std_14nmC2

array([ 6.26824043, 11.83826084,  3.67364501,  3.30133375,  5.6095456 ])

In [182]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, mean_09nmC2, width, yerr=std_09nmC2, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_11nmC2, width, yerr=std_11nmC2, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_14nmC2, width, yerr=std_14nmC2, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PEG residues Rg Unrestrained, Config 2', fontsize=16)
plt.xticks(x_pos+width, labels=aa_types, fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(0,45)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm', 'Rg = 1.4 nm'], frameon=False, fontsize=16)
plt.ylabel(r'No. of PEG residues', fontsize=16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'No. of PEG residues')

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 2 nm restrained", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_2nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()